In [72]:
import pdfplumber
import os
import re

In [125]:
path = '/Users/joowanlim/Desktop/Company/DOC ERP'
po_84428 = pdfplumber.open(os.path.join(path, 'po_84428.pdf'))
first_page = po_84428.pages[0]
second_page = po_84428.pages[1]
third_page = po_84428.pages[2]

In [123]:
po_84480 = pdfplumber.open(os.path.join(path, 'po_84480.pdf'))
first_page = po_84480.pages[0]
second_page = po_84480.pages[1]
third_page = po_84480.pages[2]

In [128]:
po_46154 = pdfplumber.open(os.path.join(path, 'po_46154.pdf'))
first_page = po_46154.pages[0]
second_page = po_46154.pages[1]
third_page = po_46154.pages[2]

In [131]:
len(po_46154.pages)

8

In [144]:
def read_pdf(path):
    pdf = pdfplumber.open(os.path.join(path))
    return pdf

def extract_tables(pdf):
    tables = []
    for page in pdf.pages:
        table = page.extract_tables(table_settings={})
        tables.append(table)
    return tables
po_46154_pdf = read_pdf(os.path.join(path, 'po_46154.pdf'))
po_46154_tables = extract_tables(po_46154_pdf)

In [145]:
def find_orderNumber_date_supplier(tables):
    order_number = date = supplier = ''
    for table in tables:
        for i in range(len(table)):
            if 'ORDER NR' in table[i]:
                next_row = table[i+1] if i+1 < len(table) else None
                order_number = next_row[0]
                date = next_row[2]
                supplier = next_row[4]
                return order_number, date, supplier


def find_season_buyer_paymentTerms(tables):
    season = buyer = payment_terms = ''
    for table in tables:
        for i in range(len(table)):
            if 'SEASON' in table[i]:
                next_row = table[i+1] if i+1 < len(table) else None
                season = next_row[0]
                buyer = next_row[2]
                payment_terms = next_row[4]
                return season, buyer, payment_terms

def find_articleNo(tables):
    articleNo = ''
    for table in tables:
        for row in table:
            for cell in row:
                if cell and re.search(r"ARTICLE.*", str(cell)):
                    parts = cell.split("\n")
                    return parts[1]

def find_compositionInformations(tables):
    composition_informations = None
    for table in tables:
        for i in range(len(table)):
            if "COMPOSITION'S INFORMATION" in table[i]:
                next_row = table[i+1] if i+1 < len(table) else None
                composition_informations = ''.join(next_row)
                return composition_informations
        

def find_careInstructions(tables):
    care_instructions = None
    for table in tables:
        for i in range(len(table)):
            if 'CARE INSTRUCTIONS' in table[i]:
                next_row = table[i+1] if i+1 < len(table) else None
                care_instructions = ''.join(next_row)
                return care_instructions


def find_totalOrders(tables):
    total_order = []
    capture = False
    for table in tables:
        for i in range(len(table)):
            if 'TOTAL ORDER' in table[i]:
                capture = True
                continue

            if "TOTAL" in table[i]:
                total_order.append(table[i])
                capture = False

            if capture:
                total_order.append(table[i])
    return total_order


def find_from_first_page(first_page_tables):
    order_number, date, supplier = find_orderNumber_date_supplier(first_page_tables)
    season, buyer, payment_terms = find_season_buyer_paymentTerms(first_page_tables)
    article_no = find_articleNo(first_page_tables)
    composition_informations = find_compositionInformatoins(first_page_tables)
    care_instructions = find_careInstructions(first_page_tables)
    total_order = find_totalOrders(first_page_tables)
    first_page_dict = {"ORDER NR": order_number,
                       "ORDER DATE": date,
                       "SUPPLIER": supplier,
                       "SEASON": season,
                       "BUYER": buyer,
                       "PAYMENT TERMS": payment_terms,
                       "ARTICLE NO": article_no,
                       "COMPOSITION INFORMATION": composition_informations,
                       "CARE INSTRUCTIONS": care_instructions,
                       "TOTAL ORDER TABLE": total_order}
    return first_page_dict
find_from_first_page(po_46154_tables[0])

{'ORDER NR': '46154-D',
 'ORDER DATE': '10/09/2024',
 'SUPPLIER': 'GUOWOO INTERNATIONAL CO.,LTD - 05536',
 'SEASON': 'W 2024',
 'BUYER': '1320',
 'PAYMENT TERMS': 'TRADE FINANCE 90 DAYS (FROM SHIPPING DATE)*',
 'ARTICLE NO': '5536/200',
 'COMPOSITION INFORMATION': 'COMPOSITION WEIGHTED COLORS 712-ECRU, 809-MEDIUGREY, 610-AUBERGINE\n53% ACRYLIC\n27% CONVENTIONAL ACRYLIC\n26% RECYCLED ACRYLIC\n40% POLYESTER\nOUTER SHELL 40% RECYCLED POLYESTER FILAMENT\n5% WOOL\n5% RESPONSIBLE WOOL (RWS)\n2% ELASTANE\n2% CONVENTIONAL ELASTANE FILAMENT',
 'CARE INSTRUCTIONS': 'MACHINE WASH UP TO 30ºC/86ºF GENTLE CYCLE || DO NOT BLEACH || IRON UP TO 110ºC/230ºF || DO NOT DRY CLEAN || DO NOT TUMBLE DRY',
 'TOTAL ORDER TABLE': [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
  ['610 - AUBERGINE', '22,980', '64,795', '53,415', '22,310', '163,500'],
  ['712 - ECRU', '22,800', '69,605', '51,035', '19,560', '163,000'],
  ['809 - MEDIUGREY', '17,220', '52,495', '38,525', '14,760', '123,000'],
  ['TOTAL', '63,000', '186,

In [185]:
def find_partial_orders(partial_orders, tables):
    logistic_order = delivery = incoterm = origin = transport_mode = presentation_type = cost_price = handover_date = ''
    for table in tables:
        capture = False
        partial_order_table = []
        
        for i in range(len(table)):
            row = table[i]
            row_str = " ".join(str(c) for c in row if c)
            
            if 'LOGISTIC ORDER' in row_str:
                next_row = table[i+1] if i+1 < len(table) else None
                logistic_order = next_row[0]
                delivery = next_row[2]
                incoterm = next_row[4]
                origin = next_row[6]
                handover_date = next_row[8]
                transport_mode = next_row[10]
                presentation_type = next_row[12]
            if "COLOUR" in row_str:
                capture = True
                partial_order_table.append(row)
                continue

            if re.search(r"^\s*COST PRICE\s*:\s*([0-9.,]+)\s*([A-Z]{3})?\s*$", row_str, flags=re.IGNORECASE):
                parts = row_str.split(":")
                if len(parts) > 1:
                    cost_price = parts[1].strip()
                capture = False
                continue
                
            if capture:
                partial_order_table.append(row)
                
            
        partial_orders.update({
            logistic_order: {
                "info": {
                    'delivery_no': delivery,
                    'incoterm': incoterm,
                    'origin': origin,
                    'handover date': handover_date,
                    'transport mode': transport_mode,
                    'presentation type': presentation_type,
                    'cost price': cost_price
                },
                "table": partial_order_table
            }
            })
    return partial_orders



In [186]:
partial_orders = {}
for table in po_46154_tables:
    find_partial_orders(partial_orders, table)
partial_orders

{'': {'info': {'delivery_no': '',
   'incoterm': '',
   'origin': '',
   'handover date': '',
   'transport mode': '',
   'presentation type': '',
   'cost price': ''},
  'table': []},
 '46154-D / 1': {'info': {'delivery_no': '1ST',
   'incoterm': 'FOB',
   'origin': 'CN-SHANGHAI',
   'handover date': '14/10/2024',
   'transport mode': 'PLANE',
   'presentation type': 'BOX',
   'cost price': '5.80 USD'},
  'table': [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
   ['712 - ECRU', '4,200', '12,810', '9,390', '3,600', '30,000'],
   ['TOTAL', '4,200', '12,810', '9,390', '3,600', '30,000']]},
 '46154-D / 3': {'info': {'delivery_no': '2ND',
   'incoterm': 'FOB',
   'origin': 'CN-SHANGHAI',
   'handover date': '14/10/2024',
   'transport mode': 'PLANE',
   'presentation type': 'BOX',
   'cost price': '5.80 USD'},
  'table': [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
   ['809 - MEDIUGREY', '2,800', '8,540', '6,260', '2,400', '20,000'],
   ['TOTAL', '2,800', '8,540', '6,260', '2,400', '20,000']]},


In [184]:
po_46154_tables[5]

[[['LOGISTIC ORDER',
   '',
   'DELIVERY',
   '',
   'INCOTERM',
   '',
   'FROM',
   '',
   'HANDOVER DATE',
   '',
   'TRANSPORT MODE',
   '',
   'PRESENTATION TYPE'],
  ['46154-D / 12',
   '',
   '13TH',
   '',
   'FOB',
   '',
   'CN-SHANGHAI',
   '',
   '19/11/2024',
   '',
   'PLANE',
   '',
   'BOX']],
 [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
  ['712 - ECRU', '2,100', '6,405', '4,695', '1,800', '15,000'],
  ['809 - MEDIUGREY', '2,800', '8,541', '6,259', '2,400', '20,000'],
  ['TOTAL', '4,900', '14,946', '10,954', '4,200', '35,000'],
  ['COST PRICE: 5.80 USD', None, None, None, None, None],
  ['', None, None, None, None, None]],
 [['LOGISTIC ORDER',
   '',
   'DELIVERY',
   '',
   'INCOTERM',
   '',
   'FROM',
   '',
   'HANDOVER DATE',
   '',
   'TRANSPORT MODE',
   '',
   'PRESENTATION TYPE'],
  ['46154-D / 18',
   '',
   '14TH',
   '',
   'FOB',
   '',
   'CN-SHANGHAI',
   '',
   '22/11/2024',
   '',
   'PLANE',
   '',
   'BOX']],
 [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],

In [182]:
po_46154_tables[6]

[[['LOGISTIC ORDER',
   '',
   'DELIVERY',
   '',
   'INCOTERM',
   '',
   'FROM',
   '',
   'HANDOVER DATE',
   '',
   'TRANSPORT MODE',
   '',
   'PRESENTATION TYPE'],
  ['46154-D / 24',
   '',
   '17TH',
   '',
   'FOB',
   '',
   'CN-SHANGHAI',
   '',
   '29/11/2024',
   '',
   'PLANE',
   '',
   'BOX']],
 [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
  ['809 - MEDIUGREY', '1,260', '3,843', '2,817', '1,080', '9,000'],
  ['TOTAL', '1,260', '3,843', '2,817', '1,080', '9,000'],
  ['COST PRICE: 5.80 USD', None, None, None, None, None],
  ['', None, None, None, None, None],
  ['ITX TRADING, S.A.\nGENERAL CONDITIONS OF PURCHASE:\n1.- Purchaser and Supplier, both as identified in the Purchase Order above (the “Parties”), agree that the present General Conditions of Purchase (the “General\nConditions”) shall apply to the purchase of the goods above described (the “Goods”) and all the obligations and rights arising in connection therewith. These\nGeneral Conditions, together with the foregoing

In [181]:
temp = {}
find_partial_orders(temp, po_46154_tables[6])


{'46154-D / 24': {'info': {'delivery_no': '17TH',
   'incoterm': 'FOB',
   'origin': 'CN-SHANGHAI',
   'handover date': '29/11/2024',
   'transport mode': 'PLANE',
   'presentation type': 'BOX',
   'cost price': "1.- Purchaser and Supplier, both as identified in the Purchase Order above (the “Parties”), agree that the present General Conditions of Purchase (the “General\nConditions”) shall apply to the purchase of the goods above described (the “Goods”) and all the obligations and rights arising in connection therewith. These\nGeneral Conditions, together with the foregoing details of the purchase of the Goods (the “Purchase Order”) and the regulations, specifications or documents\nreferred to shall together be interpreted as one agreement for the purchase of the Goods. Supplier shall communicate to Purchaser the acceptance of each Purchase\nOrder within 15 days as of its date. Notwithstanding the foregoing, the Purchase Order shall be deemed accepted by Supplier whenever he performs t

[[['LOGISTIC ORDER',
   '',
   'DELIVERY',
   '',
   'INCOTERM',
   '',
   'FROM',
   '',
   'HANDOVER DATE',
   '',
   'TRANSPORT MODE',
   '',
   'PRESENTATION TYPE'],
  ['46154-D / 24',
   '',
   '17TH',
   '',
   'FOB',
   '',
   'CN-SHANGHAI',
   '',
   '29/11/2024',
   '',
   'PLANE',
   '',
   'BOX']],
 [['COLOUR', 'XS', 'S', 'M', 'L', 'Total'],
  ['809 - MEDIUGREY', '1,260', '3,843', '2,817', '1,080', '9,000'],
  ['TOTAL', '1,260', '3,843', '2,817', '1,080', '9,000'],
  ['COST PRICE: 5.80 USD', None, None, None, None, None],
  ['', None, None, None, None, None],
  ['ITX TRADING, S.A.\nGENERAL CONDITIONS OF PURCHASE:\n1.- Purchaser and Supplier, both as identified in the Purchase Order above (the “Parties”), agree that the present General Conditions of Purchase (the “General\nConditions”) shall apply to the purchase of the goods above described (the “Goods”) and all the obligations and rights arising in connection therewith. These\nGeneral Conditions, together with the foregoing